<a href="https://colab.research.google.com/github/vishalveerareddy/Sentiment-Analysis-Using-CNN/blob/master/Text_Clasification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.datasets import imdb

# set parameters:
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2



Using TensorFlow backend.


In [0]:
print('Loading data...')
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(x_train, y_train), (x_test, y_test) = imdb.load_data(path="imdb.npz",
                                                      num_words=None,
                                                      skip_top=0,
                                                      maxlen=None,
                                                      seed=113,
                                                      start_char=1,
                                                      oov_char=2,
                                                      index_from=3)
# restore np.load for future normal usage
np.load = np_load_old

print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)



Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 400)
x_test shape: (25000, 400)


In [0]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(1))
model.add(Activation('sigmoid'))



W0803 17:50:53.298698 140057190680448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0803 17:50:53.314287 140057190680448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0803 17:50:53.316537 140057190680448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0803 17:50:53.330649 140057190680448 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0803 17:50:53.339437 

Build model...


In [0]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [0]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=10,
          validation_data=(x_test, y_test))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 6s 227us/step - loss: 0.3785 - acc: 0.8320 - val_loss: 0.3686 - val_acc: 0.8392
Epoch 2/10
25000/25000 [==============================] - 6s 224us/step - loss: 0.3433 - acc: 0.8538 - val_loss: 0.3368 - val_acc: 0.8656
Epoch 3/10
25000/25000 [==============================] - 6s 224us/step - loss: 0.3103 - acc: 0.8720 - val_loss: 0.3279 - val_acc: 0.8536
Epoch 4/10
25000/25000 [==============================] - 6s 226us/step - loss: 0.2815 - acc: 0.8832 - val_loss: 0.2992 - val_acc: 0.8804
Epoch 5/10
25000/25000 [==============================] - 6s 224us/step - loss: 0.2539 - acc: 0.8974 - val_loss: 0.3078 - val_acc: 0.8822
Epoch 6/10
25000/25000 [==============================] - 6s 222us/step - loss: 0.2339 - acc: 0.9053 - val_loss: 0.2768 - val_acc: 0.8864
Epoch 7/10
25000/25000 [==============================] - 6s 222us/step - loss: 0.2156 - acc: 0.9151 - val_loss: 0.3097 - 

In [0]:
model.predict_classes(x_test)

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [1]], dtype=int32)

In [0]:
y_test

array([0, 1, 1, ..., 0, 0, 0])

In [0]:
model.evaluate(x_test,y_test)

25000/25000 [==============================] - 1s 56us/step


[0.2791573500394821, 0.89036]

In [0]:
model.metrics_names

['loss', 'acc']